## 1, Import Library

In [ ]:
from machine_lib import * 

## 2, 登录
<div style="margin-left: 20px;">
1, 在machine_lib文件的login方法中填写用户名和密码后保存，然后来到本文件Restart Kernal后重新import machine_lib后才在本文件生效
</div>

<div style="margin-left: 20px;">
2, 打印INVALID_CREDENTAIL即登录失败，打印自己的user_id信息才是登录成功。
</div>

In [ ]:
s = login()

## 3, 获取数据字段
<div style="margin-left: 20px;">
在官网Data页面中显示的为自己目前有权限的数据集，在数据集Description面板下可以看到dataset_id
</div>

In [ ]:
df = get_datafields(s, dataset_id = 'analyst4', region='USA', universe='TOP3000', delay=1)
df

## 4，数据字段预处理

<div style="margin-left: 20px;">
1, matrix, vector 数据类型
</div>

<div style="margin-left: 20px;">
2, ts_backfill 回填缺失值，提高数据Coverage 
</div>

<div style="margin-left: 20px;">
2, winsorize 去极值
</div>

In [ ]:
pc_fields = process_datafields(df)
len(pc_fields)

## 5，Alpha Factory 
<div style="margin-left: 20px;">
在factory方法中将数据字段与操作符组装成alpha表达式
</div>

In [ ]:
first_order = first_order_factory(pc_fields, ts_ops)
print(first_order[:10])
print("生产表达式数量: %s"%len(first_order))

## 6, 回测前载入

<div style="margin-left: 20px;">
1, alpha表达式与初始decay配对
</div>

<div style="margin-left: 20px;">
2, random shuffle 
</div>

<div style="margin-left: 20px;">
2, Load task pool数据结构
</div>


In [ ]:
# 赋予alpha表达式一个初始decay
init_decay = 6
fo_alpha_list = []
for alpha in first_order:
    fo_alpha_list.append((alpha, init_decay))

# 随机采样快速评估一个数据集的潜力
random.shuffle(fo_alpha_list)

print("数量: %s"%len(fo_alpha_list))
print(fo_alpha_list[:5])

In [ ]:
# Load alphas to task pools
fo_pools = load_task_pool_single(fo_alpha_list, 3)
print(fo_pools[0])

## 7, 回测

In [ ]:
# Simulate First Order
single_simulate(fo_pools, "SUBINDUSTRY", "USA", "TOP3000", 0)

## 8, 筛选Alpha


<div style="margin-left: 20px;">
1, get_alpha：截取有潜力提升表现至可以提交的alpha进入下一阶
</div>

<div style="margin-left: 20px;">
2, 剪枝Prune：精减相似alpha，提高回测资源利用率
</div>


In [ ]:
## get promising alphas to improve in the next order
fo_tracker = get_alphas("02-27", "02-28", 1, 0.7, "USA", 100, "track")
print(len(fo_tracker))

#### Prune 剪枝

In [ ]:
fo_layer = prune(fo_tracker, 'anl4', 5)

# 剪枝后数量
print(len(fo_layer))

## 9, 二阶提升
### ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [ ]:
so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
    for alpha in get_group_second_order_factory([expr], group_ops, "USA"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)

print(len(so_alpha_list))
print(so_alpha_list[:3])

### Simulate second order

In [ ]:
so_pools = load_task_pool_single(so_alpha_list, 3)
single_simulate(so_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## 10，三阶提升
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [ ]:
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas("02-27", "02-28", 1.3, 0.8, "USA", 200, "track")

so_layer = prune(so_tracker, 'anl4', 5)
th_alpha_list = []

for expr, decay in so_layer:
    for alpha in trade_when_factory("trade_when",expr,"USA"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))

### Simulate Third Order

In [ ]:
# Simulate third order
th_pools = load_task_pool_single(th_alpha_list, 3)
single_simulate(th_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 3)

## 11, 获取可提交的Alpha

<div style="margin-left: 20px;">
1, 拉取sharpe,fitness达到提交要求的alpha
</div>

<div style="margin-left: 20px;">
2, Check Submission：检查其他Test是否达到要求
</div>

<div style="margin-left: 20px;">
2, view_alphas 对可以提交的alpha进行排序
</div>


In [ ]:
# 1.25 sharpe, 1 fitness, "submit"参数
th_tracker = get_alphas("02-27", "02-28", 1.25, 1, "USA", 200, "submit")

In [ ]:
## 将get的alpha的id取出至stone_bag，用api check submission
stone_bag = []
for alpha in th_tracker:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []
check_submission(stone_bag, gold_bag, 0)

In [ ]:
# 打印可提交的alpha信息并按sharpe排序，在网页上找到alpha手动提交
view_alphas(gold_bag)

## 12, 微调可以提交的alpha

<div style="margin-left: 20px;">
1, 得到更好的表现
</div>
<div style="margin-left: 40px;">
调整中性化，操作符参数，Decay
</div>

<div style="margin-left: 20px;">
2, Alpha质量评估
</div>

<div style="margin-left: 40px;">
performance comparison，turnover，margin
</div>

<div style="margin-left: 20px;">
3, 鲁棒性评估，防止过拟合
</div>

<div style="margin-left: 40px;">
更改中性化，Rank，Binary Test...
</div>

### Appendix

In [ ]:
# 模板构建Factory实例

def template_factory(sent_fields, option_fields):
    alpha_list = []
    for sent_field in sent_fields:
        for opt_field in option_fields:
            alpha_list.append("log(1+sigmoid(ts_zscore(%s,30))*sigmoid(ts_zscore(%s,30))"%(sent_field, opt_field))
    return alpha_list 

opt_df = get_datafields(s, dataset_id = 'option8', region='USA', universe='TOP3000', delay=1)
opt_fields = opt_df[opt_df['type'] == "MATRIX"]["id"].tolist()
print(opt_fields)

sent_df = get_datafields(s, dataset_id = 'sentiment1', region='USA', universe='TOP3000', delay=1)
sent_fields = sent_df[sent_df['type'] == "MATRIX"]["id"].tolist()
print(sent_fields)

alpha_list = template_factory(sent_fields, opt_fields)
print(alpha_list)